In [1]:
# !pip install llama-index llama-index-llms-ollama

In [2]:
# !pip install llama-index-embeddings-huggingface

In [3]:
from llama_index.embeddings.huggingface import HuggingFaceEmbedding

In [4]:
from llama_index.llms.ollama import Ollama
from llama_index.embeddings.huggingface import HuggingFaceEmbedding
from llama_index.core import Settings
import nest_asyncio

nest_asyncio.apply()

# Setup the local LLM via Ollama using the model "qwq:32b"
Settings.llm = Ollama(model="qwq:32b", request_timeout=30.0)

# Setup the embedding model using the local model directory.
Settings.embed_model = HuggingFaceEmbedding(model_name="sentence-transformers/all-MiniLM-L6-v2")

In [5]:
from llama_index.core import Settings, SimpleDirectoryReader, VectorStoreIndex
from llama_index.core.node_parser import SentenceSplitter
# -----------------------------------------------------------
# Create a Sentence Splitter for document parsing
# -----------------------------------------------------------
splitter = SentenceSplitter(chunk_size=1024)

# -----------------------------------------------------------
# Helper Function: Create Query Engine from a PDF file
# -----------------------------------------------------------
def create_query_engine(pdf_file_path: str):
    # Read the document(s)
    documents = SimpleDirectoryReader(input_files=[pdf_file_path]).load_data()
    # Parse documents into nodes using the sentence splitter
    nodes = splitter.get_nodes_from_documents(documents)
    # Create the vector store index
    index = VectorStoreIndex(nodes)
    # Return the query engine for the index
    return index.as_query_engine()

# -----------------------------------------------------------
# Setup Document Indexes for Vector Search
# -----------------------------------------------------------
# Aeroflow Specification Document
aeroflow_query_engine = create_query_engine("AeroFlow_Specification_Document.pdf")

# EchoSprint Specification Document
ecosprint_query_engine = create_query_engine("EcoSprint_Specification_Document.pdf")

In [7]:
from llama_index.core.tools import QueryEngineTool
from llama_index.core.query_engine.router_query_engine import RouterQueryEngine
from llama_index.core.selectors import LLMSingleSelector

# Create a query engine tool for Aeroflow specifications
aeroflow_tool = QueryEngineTool.from_defaults(
    query_engine=aeroflow_query_engine,
    name="Aeroflow specifications",
    description="Contains information about Aeroflow: Design, features, technology, maintenance, warranty"
)

# Create a query engine tool for EcoSprint specifications
ecosprint_tool = QueryEngineTool.from_defaults(
    query_engine=ecosprint_query_engine,
    name="EcoSprint specifications",
    description="Contains information about EcoSprint: Design, features, technology, maintenance, warranty"
)

# Create a Router Agent to select the appropriate tool based on the query
router_agent = RouterQueryEngine(
    selector=LLMSingleSelector.from_defaults(),
    query_engine_tools=[aeroflow_tool, ecosprint_tool],
    verbose=True
)

In [8]:
#Ask a question about NoSQL
response = router_agent.query("What colors are available for AeroFlow?")
print("\nResponse: ",str(response))

Selecting query engine 0: The question asks about AeroFlow's available colors. Choice (1) specifically references AeroFlow's 'Design' category, which would logically include color options as part of the product's design details..

Response:  <think>
Okay, let me look at the context provided. The user is asking about the available colors for AeroFlow.

Looking through the Design Specifications section under Exterior Design, it mentions the colors Coastal Blue, Sunset Orange, and Pearl White. I need to make sure these are the only ones listed. There's no mention of other colors on the second page either. So the answer should list those three colors without referencing the context directly.
</think>

AeroFlow is available in Coastal Blue, Sunset Orange, and Pearl White.


In [9]:
response = router_agent.query("What colors are available for EcoSprint?")
print("\nResponse: ",str(response))

Selecting query engine 1: The question asks about EcoSprint's available colors. Since item (2) specifically refers to EcoSprint and includes 'design' as one of its categories, design likely encompasses color options. Item (1) pertains to Aeroflow, which is unrelated to the queried product..

Response:  <think>
Okay, let's see. The user is asking about the available colors for the EcoSprint.

Looking at the context provided, in the Design Specifications section under Exterior Design, it mentions the colors Midnight Black, Ocean Blue, and Pearl White. I need to make sure those are the ones listed. 

Wait, the page labels show that the first page has the exterior colors. The user wants just the answer without referencing the context directly. So I should list those three colors exactly as stated. No other colors are mentioned here. Let me double-check if there's any other mention of colors elsewhere in the provided pages. Scanning through the rest, sections 1 to 7 don't mention any other 